# CRAG

### Workflow:

1. Ingestion de datos
2. Retrieval
3. Evaluación de Relevancia*
4. Extracción de Relevancia
5. Transformación de la consulta y búsqueda en la web (tavily)
6. Generación de la respuesta

### Eventos

 1. `PrepEvent`: El indice y otros objetos estan preparados.
 2. `RetrieveEvent`: El evento tiene informacion de nodos recuperados.
 3. `RelevanceEvalEvent`: El contiene una lista de resultados de la evaluación de relevancia.
 4. `TextExtractEvent`: Contiene una cadena con el texto relevante.
 5. `QueryEvent`: Contiene ambos: texto relevante y texto de la busqueda.

In [1]:
from llama_index.core.workflow import Event
from llama_index.core.schema import NodeWithScore

/Users/jose.laruta/dev/topics/ai-topics-2-2025/.venv/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:2249: UnsupportedFieldAttributeWarning: The 'validate_default' attribute with value True was provided to the `Field()` function, which has no effect in the context it was used. 'validate_default' is field-specific metadata, and can only be attached to a model field using `Annotated` metadata or by assignment. This may have happened because an `Annotated` type alias using the `type` statement was used, or if the `Field()` function was attached to a single member of a union type.
  warnings.warn(


In [2]:
class PrepEvent(Event):
    """Prep event (prepares for retrieval)."""
    pass

class RetrieveEvent(Event):
    """Retrieve event (gets retrieved nodes)."""
    retrieved_nodes: list[NodeWithScore]

class RelevanceEvalEvent(Event):
    """Relevance evaluation event (gets results of relevance evaluation)."""
    relevant_results: list[str]

class TextExtractEvent(Event):
    """Text extract event. Extracts relevant text and concatenates."""
    relevant_text: str

class QueryEvent(Event):
    """Query event. Queries given relevant text and search text."""
    relevant_text: str
    search_text: str

In [3]:
from llama_index.core.workflow import (
    Workflow,
    step,
    Context,
    StartEvent,
    StopEvent,
)
from llama_index.core import (
    VectorStoreIndex,
    Document,
    PromptTemplate,
    SummaryIndex,
)

In [4]:
from llama_index.tools.tavily_research.base import TavilyToolSpec
from llama_index.core.base.base_retriever import BaseRetriever
from llama_index.llms.openai import OpenAI

In [5]:
DEFAULT_RELEVANCY_PROMPT_TEMPLATE = PromptTemplate(
    template="""As a grader, your task is to evaluate the relevance of a document retrieved in response to a user's question.

    Retrieved Document:
    -------------------
    {context_str}

    User Question:
    --------------
    {query_str}

    Evaluation Criteria:
    - Consider whether the document contains keywords or topics related to the user's question.
    - The evaluation should not be overly stringent; the primary objective is to identify and filter out clearly irrelevant retrievals.

    Decision:
    - Assign a binary score to indicate the document's relevance.
    - Use 'yes' if the document is relevant to the question, or 'no' if it is not.

    Please provide your binary score ('yes' or 'no') below to indicate the document's relevance to the user question."""
)


In [33]:
DEFAULT_TRANSFORM_QUERY_TEMPLATE = PromptTemplate(
    template="""Your task is to refine a query to ensure it is highly effective for retrieving relevant search results. \n
    Analyze the given input to grasp the core semantic intent or meaning. \n
    Original Query:
    \n ------- \n
    {query_str}
    \n ------- \n
    Your goal is to rephrase or enhance this query to improve its search performance. Ensure the revised query is concise and directly aligned with the intended search objective. \n
    Respond with the optimized query only:"""
)

In [24]:
class CorrectiveRAGWorkflow(Workflow):
    @step
    async def ingest(self, ctx: Context, ev: StartEvent) -> StopEvent | None:
        """Ingest step (for ingesting docs and initializing index)."""
        documents: list[Document] | None = ev.get("documents")

        if documents is None:
            return None

        index = VectorStoreIndex.from_documents(documents)
        return StopEvent(result=index)
    
    @step
    async def prepare_for_retrieval(
        self, ctx: Context, ev: StartEvent
    ) -> PrepEvent | None:
        """Prepare for retrieval."""

        query_str: str | None = ev.get("query_str")
        retriever_kwargs: dict | None = ev.get("retriever_kwargs", {})

        if query_str is None:
            return None

        tavily_ai_apikey: str | None = ev.get("tavily_ai_apikey")
        index = ev.get("index")

        llm = OpenAI(model="gpt-4")

        await ctx.store.set("llm", llm)
        await ctx.store.set("index", index)
        await ctx.store.set(
            "tavily_tool", TavilyToolSpec(api_key=tavily_ai_apikey)
        )

        await ctx.store.set("query_str", query_str)
        await ctx.store.set("retriever_kwargs", retriever_kwargs)

        return PrepEvent()
    
    @step
    async def retrieve(
        self, ctx: Context, ev: PrepEvent
    ) -> RetrieveEvent | None:
        """Retrieve the relevant nodes for the query."""
        query_str = await ctx.store.get("query_str")
        retriever_kwargs = await ctx.store.get("retriever_kwargs")

        if query_str is None:
            return None

        index = await ctx.store.get("index", default=None)
        tavily_tool = await ctx.store.get("tavily_tool", default=None)
        if not (index or tavily_tool):
            raise ValueError(
                "Index and tavily tool must be constructed. Run with 'documents' and 'tavily_ai_apikey' params first."
            )

        retriever: BaseRetriever = index.as_retriever(**retriever_kwargs)
        result = retriever.retrieve(query_str)
        print(f"=====> retrieved: {len(result)} nodes: {[(i, r.score) for i,r in enumerate(result)]}")
        await ctx.store.set("retrieved_nodes", result)
        await ctx.store.set("query_str", query_str)
        return RetrieveEvent(retrieved_nodes=result)
    
    @step
    async def eval_relevance(
        self, ctx: Context, ev: RetrieveEvent
    ) -> RelevanceEvalEvent:
        """Evaluate relevancy of retrieved documents with the query."""
        retrieved_nodes = ev.retrieved_nodes
        query_str = await ctx.store.get("query_str")

        relevancy_results = []
        for node in retrieved_nodes:
            llm = await ctx.store.get("llm")
            resp = await llm.acomplete(
                DEFAULT_RELEVANCY_PROMPT_TEMPLATE.format(
                    context_str=node.text, query_str=query_str
                )
            )
            relevancy_results.append(resp.text.lower().strip())
        print(f"=====> relevancies: {relevancy_results}")
        await ctx.store.set("relevancy_results", relevancy_results)
        return RelevanceEvalEvent(relevant_results=relevancy_results)
    
    @step
    async def extract_relevant_texts(
        self, ctx: Context, ev: RelevanceEvalEvent
    ) -> TextExtractEvent:
        """Extract relevant texts from retrieved documents."""
        retrieved_nodes = await ctx.store.get("retrieved_nodes")
        relevancy_results = ev.relevant_results

        relevant_texts = [
            retrieved_nodes[i].text
            for i, result in enumerate(relevancy_results)
            if result == "yes"
        ]
        print(f"=====> relevant_texts: {len(relevant_texts)}")
        result = "\n".join(relevant_texts)
        return TextExtractEvent(relevant_text=result)
    
    @step
    async def transform_query(
        self, ctx: Context, ev: TextExtractEvent
    ) -> QueryEvent:
        """Search the transformed query with Tavily API."""
        relevant_text = ev.relevant_text
        relevancy_results = await ctx.store.get("relevancy_results")
        query_str = await ctx.store.get("query_str")

        # If any document is found irrelevant, transform the query string for better search results.
        if "no" in relevancy_results:
            llm = await ctx.store.get("llm")
            resp = await llm.acomplete(
                DEFAULT_TRANSFORM_QUERY_TEMPLATE.format(query_str=query_str)
            )
            transformed_query_str = resp.text
            print(f"=====> transformed query: {transformed_query_str}")
            # Conduct a search with the transformed query string and collect the results.
            tavily_tool = await ctx.store.get("tavily_tool")
            search_results = tavily_tool.search(
                transformed_query_str, max_results=5
            )
            search_text = "\n".join([result.text for result in search_results])
            print(f"=====> search text: {search_text}")
        else:
            search_text = ""

        return QueryEvent(relevant_text=relevant_text, search_text=search_text)
    
    @step
    async def query_result(self, ctx: Context, ev: QueryEvent) -> StopEvent:
        """Get result with relevant text."""
        relevant_text = ev.relevant_text
        search_text = ev.search_text
        query_str = await ctx.store.get("query_str")

        documents = [Document(text=relevant_text + "\n" + search_text)]
        index = SummaryIndex.from_documents(documents)
        query_engine = index.as_query_engine()
        result = query_engine.query(query_str)
        return StopEvent(result=result)

In [7]:
from llama_index.utils.workflow import draw_all_possible_flows

draw_all_possible_flows(
    CorrectiveRAGWorkflow,
    filename="CRAG.html",
    # Optional, can limit long event names in your workflow
    # Can help with readability
    # max_label_length=10,
)

CRAG.html


## Corriendo el workflow

In [12]:
from llama_index.core import SimpleDirectoryReader
from dotenv import load_dotenv
load_dotenv()

True

In [29]:
documents = SimpleDirectoryReader("./data").load_data()

In [34]:
workflow = CorrectiveRAGWorkflow(verbose=True)

In [31]:
index = await workflow.run(documents=documents)

Running step ingest


2025-10-17 08:44:03,012 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-10-17 08:44:04,668 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


Step ingest produced event StopEvent
Running step prepare_for_retrieval
Step prepare_for_retrieval produced no event


In [35]:
import os
from IPython.display import Markdown, display

tavily_ai_api_key = os.getenv("TAVILY_API_KEY")

response = await workflow.run(
    query_str="Cuantos embarazos adolescentes se registraron en 2022 en bolivia?",
    index=index,
    tavily_ai_apikey=tavily_ai_api_key,
)

Running step ingest
Step ingest produced no event
Running step prepare_for_retrieval
Step prepare_for_retrieval produced event PrepEvent
Running step retrieve


2025-10-17 08:45:05,053 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


=====> retrieved: 2 nodes: [(0, 0.8964927840661618), (1, 0.8649018795244673)]
Step retrieve produced event RetrieveEvent
Running step eval_relevance


2025-10-17 08:45:06,151 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-17 08:45:07,017 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=====> relevancies: ['yes', 'no']
Step eval_relevance produced event RelevanceEvalEvent
Running step extract_relevant_texts
=====> relevant_texts: 1
Step extract_relevant_texts produced event TextExtractEvent
Running step transform_query


2025-10-17 08:45:08,102 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=====> transformed query: "Statistics on teenage pregnancies in Bolivia in 2022"
=====> search text: The maternal mortality ratio in 2020 was estimated at 160.9 deaths per 100 000 live births, representing a 43.3% reduction compared to the estimated value in 2000 (Figure 5). In relation to fertility, it is estimated that in 2024 women had an average of 2.5 children throughout their reproductive lives. In the specific case of adolescent fertility, there was a 31.7% decrease, from 94.3 live births per 1000 women aged 15 to 19 years in 2000 to 64.4 in 2024. In 2022, 98.2% of births were attended [...] by skilled birth personnel. [...] Between 2003 and 2022, infant mortality in Bolivia (the Plurinational State of) decreased from 54 to 20.44 deaths per 1000 live births, a decrease of 62.1% (Figure 4). The percentage of low-weight births (less than 2500 g) increased from 4.0% to 5.4% between 2001 and 2022. While exclusive breastfeeding in the child population up to 6 months of age was 53.6% 

2025-10-17 08:45:14,020 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Step query_result produced event StopEvent


In [36]:
display(Markdown(str(response)))

Se registraron un total de 35,250 casos de embarazos adolescentes en Bolivia en 2022.

In [37]:
response = await workflow.run(
    query_str="Cuantos embarazos adolescentes se registraron en 2024 en bolivia?",
    index=index,
    tavily_ai_apikey=tavily_ai_api_key,
)
display(Markdown(str(response)))

Running step ingest
Step ingest produced no event
Running step prepare_for_retrieval
Step prepare_for_retrieval produced event PrepEvent
Running step retrieve


2025-10-17 08:46:55,789 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


=====> retrieved: 2 nodes: [(0, 0.894294828274153), (1, 0.8654738292205189)]
Step retrieve produced event RetrieveEvent
Running step eval_relevance


2025-10-17 08:46:57,478 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-10-17 08:46:58,664 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=====> relevancies: ['no', 'no']
Step eval_relevance produced event RelevanceEvalEvent
Running step extract_relevant_texts
=====> relevant_texts: 0
Step extract_relevant_texts produced event TextExtractEvent
Running step transform_query


2025-10-17 08:46:59,701 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


=====> transformed query: "Statistics on teenage pregnancies in Bolivia in 2024"
=====> search text: | Belize | 23·7% (18·7–29·6) | 24·1% (19·4–29·5) | 24·3% (19·7–29·7) | 21·0% (17·6–24·8) | 19·7% (16·4–23·5) | 17·3% (14·5–20·5) | −1·5 (−2·5 to −0·5) |
| Bolivia | 18·6% (15·4–22·4) | 15·5% (13·2–18·3) | 15·6% (13·5–18·0) | 17·9% (15·9–20·2) | 15·8% (13·8–18·0) | 17·1% (14·9–19·5) | 0·0 (−0·6 to 0·6) |
| Colombia | 14·1% (12·7–15·6) | 16·6% (15·0–18·2) | 19·4% (17·5–21·5) | 21·8% (20·0–23·8) | 20·0% (18·2–22·0) | 19·5% (17·9–21·1) | 1·2 (0·8 to 1·5) | [...] | Jamaica | 27·6% (23·3–32·4) | 25·6% (21·6–30·1) | 21·9% (18·8–25·4) | 23·7% (19·7–28·2) | 19·5% (15·9–23·7) | 14·9% (12·2–18·0) | −2·4 (−3·3 to −1·6) |
| Mexico | 13·0% (10·0–16·6) | 17·2% (14·0–20·8) | 13·9% (11·7–16·5) | 18·7% (15·2–22·9) | 16·6% (13·5–20·2) | 20·5% (17·8–23·5) | 1·3 (0·5 to 2·0) |
| Paraguay | 14·0% (10·8–17·8) | 16·8% (13·4–20·8) | 19·6% (16·2–23·5) | 16·8% (13·8–20·2) | 15·3% (12·7–18·2) | 14·8% (12·5–17·4) |

2025-10-17 08:47:04,622 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Step query_result produced event StopEvent


14.34%